# Apophis: 2029 B-plane (5th-order, 7-DOF jet transport)

In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using JLD, PlanetaryEphemeris, Apophis, SPICE

In [ ]:
using TaylorIntegration

In [ ]:
#using LinearAlgebra: dot, cross, norm, diag
using LinearAlgebra
using Dates, Plots

In [ ]:
using Statistics

In [ ]:
dq = set_variables("ξ", order=5, numvars=7)

In [ ]:
fnamesseph = joinpath(Apophis.jldephpath, "sseph343ast016_p30y_et_J2000.jld")
@time ss16asteph = load(fnamesseph, "ss16ast_eph");

In [ ]:
txgS1_file = "txgS1.jld" # p21y: sep 2008 - sep 2029
tvS1 = load(txgS1_file, "tvS1")
xvS1 = load(txgS1_file, "xvS1")
gvS1 = load(txgS1_file, "gvS1");

In [ ]:
x_Gamma_file = "x_Gamma.jld"
x_O7 = load(x_Gamma_file, "x_O7")
Γ_O7 = load(x_Gamma_file, "Γ_O7")
x_O6 = load(x_Gamma_file, "x_O6")
Γ_O6 = load(x_Gamma_file, "Γ_O6")
x_R = load(x_Gamma_file, "x_R")
Γ_R = load(x_Gamma_file, "Γ_R")
x_OR7 = load(x_Gamma_file, "x_OR7")
Γ_OR7 = load(x_Gamma_file, "Γ_OR7")
x_OR6 = load(x_Gamma_file, "x_OR6")
Γ_OR6 = load(x_Gamma_file, "Γ_OR6");

In [ ]:
x_OR7

In [ ]:
@show x_OR7.*vcat(1e-8ones(6), 1e-14)

In [ ]:
# updated orbital fit including ALL astrometry- 2004-march 8 2021 + 2 new delays + 1 new Doppler
x_OR7_NEW = [4.3694215245970724, 7.987213945792604, 3.6413243281741003, -0.13423783109395293, 0.06725563717058236, 0.04920255739296331, -2.895160537809007]

# covariance matrix at 2020-Dec-17 epoch
Γ_OR7_NEW_2020dec17 = [1.438434994105531 0.29220544936960297 -0.422571935602575 -0.0027635945769280117 0.014490875841090746 0.020538254916658012 0.008931059340899494; 0.29220544943836946 0.07044883207758046 -0.03759669704463631 -7.17936777241948e-5 0.0028433791046060157 0.003315402872272071 0.0036173074184983264; -0.42257193529095993 -0.03759669695365462 0.41000521339389573 0.003431929409710754 -0.0044188710016318826 -0.011382787939101226 0.008192801923556356; -0.0027635945738109682 -7.179367691067121e-5 0.0034319294092731946 3.008993325573492e-5 -3.0500082705440965e-5 -8.784712204748608e-5 8.11985957323578e-5; 0.014490875840515761 0.002843379103823599 -0.004418871004416342 -3.050008273410192e-5 0.0001479773957114174 0.00020867889636787495 8.679251516578333e-5; 0.020538254911043618 0.0033154028699902 -0.011382787942014604 -8.784712208450746e-5 0.0002086788963230772 0.0003979059915404698 -7.73348080102816e-5; 0.008931059352690285 0.0036173074207718706 0.008192801918289591 8.119859569612309e-5 8.679251528102434e-5 -7.733480780525257e-5 0.0007052356651251199]

# state transtition matrix from t0 = 2020-Dec-17 -> t = 2008-Sep-09
∂Φ_t_t0 = [1.5505725843757433e-7 -8.065138985501717e-7 -2.961209982390144e-7 4.9219775913785714e-5 -1.939304055732194e-7 1.2771618215815975e-6 1.4975599761788745e-7; 2.4336465215831323e-7 -1.3428903445480706e-6 -4.952323288850057e-7 8.238308767562521e-5 6.036774902879665e-7 2.2995070102395087e-6 2.409392990257591e-7; 9.362729391122228e-8 -5.184739860164263e-7 -1.8503709925721437e-7 3.1667315126975086e-5 2.2952692075606837e-8 1.4156220428495185e-6 9.302839600842665e-8; -4.386791523157768e-9 2.3957676670964306e-8 8.800050547022203e-9 -1.4566034068557096e-6 -5.094013217110899e-9 -4.229557867809726e-8 -4.3671639600439004e-9; 2.1547337838676814e-9 -1.2548611169006537e-8 -4.563565858559367e-9 7.655388603919235e-7 1.1375526682273566e-8 2.3190442360280577e-8 2.2434192921309108e-9; 6.940232950663992e-10 -4.019331677611499e-9 -1.606303970897597e-9 2.480730980098363e-7 2.0174544656497903e-9 1.3341835568794901e-8 7.220540197484644e-10; 0.0 0.0 0.0 0.0 0.0 0.0 1.0e-14]

# covariance matrix at 2008-Sep-09 epoch
Γ_OR7_NEW = diagm(vcat(1e8ones(6), 1e14))*( (∂Φ_t_t0)*Γ_OR7_NEW_2020dec17*(∂Φ_t_t0') )*diagm(vcat(1e8ones(6), 1e14))

# Cholesky factorization of covariance matrix

In [ ]:
Hermitian(Γ_OR7) - Γ_OR7

In [ ]:
norm(Hermitian(Γ_OR7) - Γ_OR7)

In [ ]:
Γ_O6_cholf = cholesky(Hermitian(Γ_O6[1:6,1:6]))
Γ_O7_cholf = cholesky(Hermitian(Γ_O7))
Γ_R_cholf = cholesky(Hermitian(Γ_R))
Γ_OR7_cholf = cholesky(Hermitian(Γ_OR7))
Γ_OR6_cholf = cholesky(Hermitian(Γ_OR6[1:6,1:6]))
Γ_OR7_NEW_cholf = cholesky(Hermitian(Γ_OR7_NEW))
;

In [ ]:
Γ_O6_cU = Γ_O6_cholf.U
Γ_O6_cL = Γ_O6_cholf.L

Γ_O7_cU = Γ_O7_cholf.U
Γ_O7_cL = Γ_O7_cholf.L

Γ_R_cU = Γ_R_cholf.U
Γ_R_cL = Γ_R_cholf.L

Γ_OR7_cU = Γ_OR7_cholf.U
Γ_OR7_cL = Γ_OR7_cholf.L

Γ_OR6_cU = Γ_OR6_cholf.U
Γ_OR6_cL = Γ_OR6_cholf.L

Γ_OR7_NEW_cU = Γ_OR7_NEW_cholf.U
Γ_OR7_NEW_cL = Γ_OR7_NEW_cholf.L;

In [ ]:
x_OR7

In [ ]:
σ_OR7 = sqrt.(diag(Γ_OR7))

In [ ]:
sqrt.(diag(Γ_OR7)) .* vcat(1e-8ones(6), 1e-14)

## $\Gamma$ matrix eigenvalues/vectors

In [ ]:
# Compute nominal initial condition in au, au/day, au/day^2
q0_OR7 = Apophis.initialcond(vcat(1e-8ones(6), 1e-14).*x_OR7)

In [ ]:
# relative scalings analogous to Milani (2005)
scaling_milani_2005 = vcat(  (1/norm(q0_OR7[1:3]))*ones(3), (1/norm(q0_OR7[4:6]))*ones(3), 1/norm(q0_OR7[7])  )

In [ ]:
# jet transport propagation scaling
jt_scaling = vcat(1e-8ones(6), 1e-14)

In [ ]:
# compute cov matrix with relative scalings
Γ_OR7_scm05 = diagm(scaling_milani_2005) * ( diagm(jt_scaling)*Γ_OR7*diagm(jt_scaling) ) * diagm(scaling_milani_2005)

In [ ]:
# compute normal matrix with relative scalings
C_OR7_scm05 = inv(Γ_OR7_scm05)

In [ ]:
# compute eigenvalues, eigenvectors of normal matrix with relative scalings
C_eig_OR7_scm05 = eigen(C_OR7_scm05)

In [ ]:
# revert eigenvector of "weak" direction to JT scaling
C_eig_OR7_scm05.vectors[:,1]./scaling_milani_2005./jt_scaling

### Generating samples from cov matrices

Confidence ellipsoids are defined by
$$
(\tilde{x}-x^\star)^T C (\tilde{x}-x^\star) = \sigma^2
$$
where $C$ is the normal matrix (essentially, the Hessian) with size $N\times N$, where $N$ is the number of degrees of freedom of the orbit determination. That is,  $N=6$ when dealing only with positions and velocities, and $N=7$ when dealing with positions, velocities and $A_2$, i.e., the Yarkovsky non-gravitational parameter. The covariance matrix is $\Gamma=C^{-1}$.


According to Bierman (1977) do this using the square-root information matrix., in order to sample an ensemble of initial conditions with zero mean and covariance $\Gamma$, we may perform a Cholesky factorization on $\Gamma$:
$$
\Gamma = L R
$$
where $L$ and $R$ are called, respectively, the left and right factors of the Cholesky factorization. The $L$ and $R$ matrices are, respectively, lower and upper triangular matrices. Now, let $w$ denote a zero-mean, unit standard-deviation random $N$-vector (i.e., an $N$-vector whose components are each a random number sampled from a normal distribution). Further, let $\hat{x}$ denote the nominal solution. Then
$$
\hat{x} + L \cdot w
$$
is a random $N$-vector with mean equal to the nominal solution and covariance $\Gamma$ (Bierman, 1977; Chodas and Yeomans, 1999).  Thus, in order to generate random samples from the covariance matrix, it suffices to multiply the left Cholesky factor by 6-vectors of normal random samples, as done below.

$\Gamma_{new} = J^T \Gamma * J$

### 6-variable, optical-only covariance matrix

In [ ]:
#Hermitian(Γ_O6)

In [ ]:
# recovering the covariance matrix from cholesky(Γ).L*randn(6)
#cov(map(x->Γ_O6_cholf.L*randn(6), 1:100000))

### 7-variable, optical+radar covariance matrix

In [ ]:
#Hermitian(Γ_OR7)

In [ ]:
# recovering the covariance matrix from cholesky(Γ).L*randn(7)
#cov(map(x->Γ_OR7_cholf.L*randn(7), 1:100000))

# 2029 B-plane

In [ ]:
#asteph_x0_s197 = load("Apophis_x0_s197_2008_2029_NOJT.jld", "asteph");
#ast_jld = load("Apophis_x0_s197_2008_2029_NOJT.jld")

In [ ]:
#ast_jld_tv = ast_jld["tv"]
#ast_jld_xv = ast_jld["xv"]
#;

In [ ]:
#ss16asteph(ast_jld_tv[1]*daysec)[nbodyind(27, 1)]

In [ ]:
#ast_jld_xv_eclip = map(x->begin
#        xas = asteph_x0_s197(x)[1:6] .- ss16asteph(x*daysec)[nbodyind(27, 1)]
#        ϵ0_deg = 84381.448/3600
#        m_eq2ecl = Rx(deg2rad(ϵ0_deg))
#        m_xv_eq2ecl = hcat(vcat(m_eq2ecl, zeros(3,3)), vcat(zeros(3,3), m_eq2ecl))
#        m_xv_eq2ecl*xas
#    end, 
#    ast_jld_tv
#)
#ast_jld_x1v_eclip = map(x->x[1], ast_jld_xv_eclip)
#ast_jld_x2v_eclip = map(x->x[2], ast_jld_xv_eclip)
#ast_jld_x3v_eclip = map(x->x[3], ast_jld_xv_eclip)

In [ ]:
#plot(2000 .+ (ast_jld_tv./yr), ast_jld_x3v_eclip)

In [ ]:
#findlast(x->x<0, ast_jld_x3v_eclip[findall(x->x>2029, 2000 .+ (ast_jld_tv./yr))])

In [ ]:
#ast_jld_x3v_eclip[findall(x->x>2029, 2000 .+ (ast_jld_tv./yr))][35-1:35+2]

In [ ]:
#findall(x->x>2029, 2000 .+ (ast_jld_tv./yr))[35]

In [ ]:
#begin
#    x = ast_jld_tv[1047]+0.792598034281(ast_jld_tv[1048]-ast_jld_tv[1047])
#    xas = asteph_x0_s197(x)[1:6] .- ss16asteph(x*daysec)[nbodyind(27, 1)]
#    ϵ0_deg = 84381.448/3600
#    m_eq2ecl = Rx(deg2rad(ϵ0_deg))
#    m_xv_eq2ecl = hcat(vcat(m_eq2ecl, zeros(3,3)), vcat(zeros(3,3), m_eq2ecl))
#    x, (m_xv_eq2ecl*xas)[3]
#end

In [ ]:
#t_2029_eclip = 10695.343129534973-ast_jld_tv[1]

In [ ]:
t_2029 = tvS1[47];

In [ ]:
t_2029()

In [ ]:
jd0 = datetime2julian(DateTime(2008,9,24))

In [ ]:
julian2datetime(constant_term(jd0+t_2029))

In [ ]:
ss16asteph.t0

In [ ]:
const Nm1 = (size(ss16asteph.x)[2]-13) ÷ 6
const N = Nm1 + 1

In [ ]:
ss16asteph.t[2]

In [ ]:
constant_term(t_2029)/365.25

In [ ]:
ss16asteph_2029 = ss16asteph(((jd0-J2000)+t_2029)*daysec)

# Earth's barycentric pos/vel at Apophis 2029 CA as a function of A2
xe_2029 = ss16asteph_2029[nbodyind(Nm1, ea)];
# Sun's barycentric pos/vel at Apophis 2029 CA as a function of A2
xs_2029 = ss16asteph_2029[nbodyind(Nm1, su)];

In [ ]:
apophis_2029 = xvS1[47,1:6];

In [ ]:
# Earth's heliocentric pos/vel at Apophis 2029 CA as a function of A2
xes_2029 = xe_2029 - xs_2029
# Apophis' heliocentric pos/vel at Apophis 2029 CA as a function of A2
xae_2029 = apophis_2029 - xe_2029;

In [ ]:
a_es_2029 = semimajoraxis(xes_2029..., μ[1], μ[4])()

In [ ]:
B_2029 = bopik(xae_2029, xes_2029);

In [ ]:
res_2029 = sqrt(xes_2029[1](x_OR7)^2 + xes_2029[2](x_OR7)^2 + xes_2029[3](x_OR7)^2)

In [ ]:
norm([B_2029.ξ(x_OR7), B_2029.ζ(x_OR7)])*RE # B 2029 [km]

# Valsecchi circles

We compute Valsecchi circles as a function of the components of the unperturbed planetocentric velocity vector $\mathbf{U}$ of Apophis in a coordinate frame $(X,Y,Z)$ such that the $Y$-axis is along the heliocentric velocity vector of the planet. It is enough to have the $U_y=U\cos\theta$ component and the norm $U=|\mathbf{U}|$, where $\theta$ is the angle between the $Y$-axis and $\mathbf{U}$. Furthermore, $\mathbf{U}$ should be in units such that the planet's heliocentric velocity is 1.

In [ ]:
res_2029

In [ ]:
B_Uy_2029 = B_2029.U.y(x_OR6)
#B_Uy_2029 = B_2029.U.y(x_OR7_NEW)
#B_Uy_2029 = B_2029.U.y()

In [ ]:
B_Unorm_2029 = B_2029.U.norm(x_OR6)
#B_Unorm_2029 = B_2029.U.norm(x_OR7_NEW)
#B_Unorm_2029 = B_2029.U.norm()

In [ ]:
R_, D_ = valsecchi_circle(B_Uy_2029, B_Unorm_2029, 7, 6)
D_Valsecchi_7_6 = D_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
R_Valsecchi_7_6 = R_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
D_Valsecchi_7_6, R_Valsecchi_7_6, D_Valsecchi_7_6-R_Valsecchi_7_6

In [ ]:
R_, D_ = valsecchi_circle(B_Uy_2029, B_Unorm_2029, 6, 5)
D_Valsecchi_6_5 = D_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
R_Valsecchi_6_5 = R_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
D_Valsecchi_6_5, R_Valsecchi_6_5, D_Valsecchi_6_5-R_Valsecchi_6_5

In [ ]:
R_, D_ = valsecchi_circle(B_Uy_2029, B_Unorm_2029, 8, 7)
D_Valsecchi_8_7 = D_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
R_Valsecchi_8_7 = R_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
D_Valsecchi_8_7, R_Valsecchi_8_7, D_Valsecchi_8_7-R_Valsecchi_8_7

In [ ]:
R_, D_ = valsecchi_circle(B_Uy_2029, B_Unorm_2029, 39, 34)
D_Valsecchi_39_34 = D_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
R_Valsecchi_39_34 = R_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
D_Valsecchi_39_34, R_Valsecchi_39_34, D_Valsecchi_39_34-R_Valsecchi_39_34

In [ ]:
R_, D_ = valsecchi_circle(B_Uy_2029, B_Unorm_2029, 39, 33)
D_Valsecchi_39_33 = D_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
R_Valsecchi_39_33 = R_/(PlanetaryEphemeris.RE)*au #/(1+0.0167)
D_Valsecchi_39_33, R_Valsecchi_39_33, D_Valsecchi_39_33-R_Valsecchi_39_33

In [ ]:
A2_JPL = -5.592840054057059
σ_A2_JPL = 2.201

In [ ]:
npoints = 3000
# note: no sigma factor needed here, since enclosing volume already contains ~3-sigma
rndvv = [randn(7) for i in 1:npoints];

In [ ]:
# sample uniformly random points on N-hypersphere
# code borrowed from http://corysimon.github.io/articles/uniformdistn-on-sphere/
function rand_sph(ndimsphere)
    v = zeros(ndimsphere)  # initialize so we go into the while loop
    while norm(v) < 0.0001
        v = randn(ndimsphere)
    end
    return v/norm(v)  # normalize to unit norm and return
end

In [ ]:
# generate `npoints` points on 7-dim hypersphere
#rndvv = [sigma_factor*rand_sph(7) for i in 1:npoints];

In [ ]:
dxv_O6 = map(x->(x_O6 .+ vcat(Γ_O6_cL*x[1:6],0.0) ), rndvv)
dxv_O7 = map(x->(x_O7 .+ Γ_O7_cL*x ), rndvv)
dxv_R = map(x->(x_R .+ Γ_R_cL*x ), rndvv)
dxv_OR7 = map(x->(x_OR7 .+ Γ_OR7_cL*x ), rndvv)
dxv_OR6 = map(x->(x_OR6 .+ vcat(Γ_OR6_cL*x[1:6],0.0) ), rndvv)
dxv_OR7_NEW = map(x->(x_OR7_NEW .+ Γ_OR7_NEW_cL*x ), rndvv)

;

In [ ]:
# optical-only solution, no Yarkovsky
B_dot_ξv_O6 = B_2029.ξ.(dxv_O6)
B_dot_ζv_O6 = B_2029.ζ.(dxv_O6)

# optical-only solution
B_dot_ξv_O7 = B_2029.ξ.(dxv_O7)
B_dot_ζv_O7 = B_2029.ζ.(dxv_O7)

# radar-only solution
B_dot_ξv_R = B_2029.ξ.(dxv_R)
B_dot_ζv_R = B_2029.ζ.(dxv_R)

# optical+radar solution
B_dot_ξv_OR7 = B_2029.ξ.(dxv_OR7)
B_dot_ζv_OR7 = B_2029.ζ.(dxv_OR7)

# optical+radar solution, no Yarkovsky
B_dot_ξv_OR6 = B_2029.ξ.(dxv_OR6)
B_dot_ζv_OR6 = B_2029.ζ.(dxv_OR6)

# optical+radar nominal solution
B_dot_ξ0 = B_2029.ξ(x_OR7)
B_dot_ζ0 = B_2029.ζ(x_OR7)

B_dot_ξv_OR7_NEW = B_2029.ξ.(dxv_OR7_NEW)
B_dot_ζv_OR7_NEW = B_2029.ζ.(dxv_OR7_NEW)
;

In [ ]:
BB_OR7 = sqrt.(B_dot_ξv_OR7.^2 + B_dot_ζv_OR7.^2)*au/RE;

In [ ]:
BB_OR7_NEW = sqrt.(B_dot_ξv_OR7_NEW.^2 + B_dot_ζv_OR7_NEW.^2)*au/RE;

In [ ]:
# impact probability a la Monte Carlo
count(x->x < B_2029.b(), BB_OR7)/length(BB_OR7)

In [ ]:
# Jacobian of transformation from initial conditions to B.ξ, B.ζ coordinates on target plane
t_x2B = hcat(TaylorSeries.gradient(B_2029.ξ), TaylorSeries.gradient(B_2029.ζ));
size(t_x2B)

In [ ]:
# transform cov matrix to B-plane coords
Γ_B_O6 = Hermitian(t_x2B(x_O6)[1:6,:]'*Γ_O6[1:6,1:6]*(t_x2B(x_O6)[1:6,:]))
# get cholesky-L factor
L_B_O6 = cholesky(Γ_B_O6).L 

# repeat for each solution and corresponding cov matrix

Γ_B_O7 = Hermitian(t_x2B(x_O7)'*Γ_O7*(t_x2B(x_O7)))
L_B_O7 = cholesky(Γ_B_O7).L

Γ_B_R = Hermitian(t_x2B(x_R)'*Γ_R*(t_x2B(x_R)))
L_B_R = cholesky(Γ_B_R).L

Γ_B_OR7 = Hermitian(t_x2B(x_OR7)'*Γ_OR7*(t_x2B(x_OR7)))
L_B_OR7 = cholesky(Γ_B_OR7).L

Γ_B_OR6 = Hermitian(t_x2B(x_OR6)[1:6,:]'*Γ_OR6[1:6,1:6]*(t_x2B(x_OR6)[1:6,:]))
L_B_OR6 = cholesky(Γ_B_OR6).L

Γ_B_OR7_NEW = Hermitian(t_x2B(x_OR7_NEW)'*Γ_OR7_NEW*(t_x2B(x_OR7_NEW)))
L_B_OR7_NEW = cholesky(Γ_B_OR7_NEW).L
;

In [ ]:
sigma_factor = 3
sigma_factor_5 = 5

In [ ]:
angv = vcat(0:0.001:2pi, 0);

In [ ]:
# sample points on 2-dim unit sphere

#rndvv_2sph = [sigma_factor*rand_sph(2) for i in 1:500]
#rndv_2sph_ξ = [rndvv_2sph[i][1] for i in 1:size(rndvv_2sph)[1]]
#rndv_2sph_ζ = [rndvv_2sph[i][2] for i in 1:size(rndvv_2sph)[1]];

# on 2-dim unit sphere (unit 1-sphere?), things are as easy as sampling the (3) unit circle...
rndv_2sph_ξ = sigma_factor*cos.(angv)
rndv_2sph_ζ = sigma_factor*sin.(angv);

rndv_2sph_ξ_5 = sigma_factor_5*cos.(angv)
rndv_2sph_ζ_5 = sigma_factor_5*sin.(angv);

In [ ]:
# then, project points from circle with radius 3 (i.e., locus of 3-sigma random points) onto B-plane coordinates

B_ξ_3σ_ell_O6 = (B_2029.ξ(x_O6) .+ L_B_O6[1,1]*rndv_2sph_ξ .+ L_B_O6[1,2]*rndv_2sph_ζ)
B_ζ_3σ_ell_O6 = (B_2029.ζ(x_O6) .+ L_B_O6[2,1]*rndv_2sph_ξ .+ L_B_O6[2,2]*rndv_2sph_ζ)

B_ξ_3σ_ell_O7 = (B_2029.ξ(x_O7) .+ L_B_O7[1,1]*rndv_2sph_ξ .+ L_B_O7[1,2]*rndv_2sph_ζ)
B_ζ_3σ_ell_O7 = (B_2029.ζ(x_O7) .+ L_B_O7[2,1]*rndv_2sph_ξ .+ L_B_O7[2,2]*rndv_2sph_ζ)

B_ξ_3σ_ell_R = (B_2029.ξ(x_R) .+ L_B_R[1,1]*rndv_2sph_ξ .+ L_B_R[1,2]*rndv_2sph_ζ)
B_ζ_3σ_ell_R = (B_2029.ζ(x_R) .+ L_B_R[2,1]*rndv_2sph_ξ .+ L_B_R[2,2]*rndv_2sph_ζ)

B_ξ_3σ_ell_OR7 = (B_2029.ξ(x_OR7) .+ L_B_OR7[1,1]*rndv_2sph_ξ .+ L_B_OR7[1,2]*rndv_2sph_ζ)
B_ζ_3σ_ell_OR7 = (B_2029.ζ(x_OR7) .+ L_B_OR7[2,1]*rndv_2sph_ξ .+ L_B_OR7[2,2]*rndv_2sph_ζ)

B_ξ_3σ_ell_OR7_NEW = (B_2029.ξ(x_OR7_NEW) .+ L_B_OR7_NEW[1,1]*rndv_2sph_ξ .+ L_B_OR7_NEW[1,2]*rndv_2sph_ζ)
B_ζ_3σ_ell_OR7_NEW = (B_2029.ζ(x_OR7_NEW) .+ L_B_OR7_NEW[2,1]*rndv_2sph_ξ .+ L_B_OR7_NEW[2,2]*rndv_2sph_ζ)

B_ξ_3σ_ell_OR6 = (B_2029.ξ(x_OR6) .+ L_B_OR6[1,1]*rndv_2sph_ξ .+ L_B_OR6[1,2]*rndv_2sph_ζ)
B_ζ_3σ_ell_OR6 = (B_2029.ζ(x_OR6) .+ L_B_OR6[2,1]*rndv_2sph_ξ .+ L_B_OR6[2,2]*rndv_2sph_ζ)

# 5-sigma
B_ξ_5σ_ell_OR7 = (B_2029.ξ(x_OR7) .+ L_B_OR7[1,1]*rndv_2sph_ξ_5 .+ L_B_OR7[1,2]*rndv_2sph_ζ_5)
B_ζ_5σ_ell_OR7 = (B_2029.ζ(x_OR7) .+ L_B_OR7[2,1]*rndv_2sph_ξ_5 .+ L_B_OR7[2,2]*rndv_2sph_ζ_5)
;

In [ ]:
plotly()
#gr()
#pgfplotsx()

In [ ]:
#scatter(
#    B_dot_ξv_OR7,
#    B_dot_ζv_OR7,
#    label="JTMC, OR7",
#    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
#)
plot(
    B_ξ_3σ_ell_OR7,
    B_ζ_3σ_ell_OR7,
    label="3-σ, OR7",
    leg=:outertopright,
    aspectratio=0.75
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7_NEW,
    B_ζ_3σ_ell_OR7_NEW,
    label="3-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_5σ_ell_OR7,
    B_ζ_5σ_ell_OR7,
    label="5-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!([B_dot_ξ0],
    [B_dot_ζ0],
    label="nominal, OR7",
    marker=:circle, ms=3.0,
)
#scatter!(
#    B_dot_ξv_OR6,
#    B_dot_ζv_OR6,
#    label="JTMC, OR6 (A2=0)",
#    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
#)
plot!(
    B_ξ_3σ_ell_OR6,
    B_ζ_3σ_ell_OR6,
    label="3-σ, OR6 (A2=0)",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!([B_2029.ξ(x_OR6)],
    [B_2029.ζ(x_OR6)],
    label="nominal, OR6 (A2=0)",
    marker=:circle, ms=3.0,
)
scatter!([B_2029.ξ(x_OR7_NEW)],
    [B_2029.ζ(x_OR7_NEW)],
    label="nominal, OR7_NEW",
    marker=:circle, ms=3.0,
)
plot!(cos.(angv), sin.(angv), label="Earth radius")
plot!(B_2029.b()*cos.(angv), B_2029.b()*sin.(angv), label="Earth cross section")
plot!(
    R_Valsecchi_8_7*cos.(angv), 
    D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv), 
    label="2037 Valsecchi circle (8:7)",
    #aspectratio=0.8,#1,
    #legend=false,
)
plot!(
    R_Valsecchi_7_6*cos.(angv), 
    D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv), 
    label="2036 Valsecchi circle (7:6)",
)
plot!(
    R_Valsecchi_39_34*cos.(angv), 
    D_Valsecchi_39_34.+R_Valsecchi_39_34*sin.(angv), 
    label="2068 Valsecchi circle (39:34)",
)
#plot!(
#    R_Valsecchi_39_33*cos.(angv), 
#    D_Valsecchi_39_33.+R_Valsecchi_39_33*sin.(angv), 
#    label="2068 Valsecchi circle (39:33)",
#)
xlabel!("B.ξ [Earth radii]")
ylabel!("B.ζ [Earth radii]")
title!("B-plane (Öpik's frame)")
xlims!(-5, 5)
xticks!(-5:5)
ylims!(-3, 9)
yticks!(-3:9)

In [ ]:
7.1422*RE, 7.6146*RE

In [ ]:
gr()

In [ ]:
scatter(
    B_dot_ξv_O7,
    B_dot_ζv_O7,
    label="JTMC, O7", 
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!(
    B_dot_ξv_R,
    B_dot_ζv_R,
    label="JTMC, R7", 
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!(
    B_dot_ξv_OR7,
    B_dot_ζv_OR7,
    label="JTMC, OR7",
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_O7,
    B_ζ_3σ_ell_O7,
    label="3-σ, O7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_R,
    B_ζ_3σ_ell_R,
    label="3-σ, R7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7,
    B_ζ_3σ_ell_OR7,
    label="3-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!([B_dot_ξ0],
    [B_dot_ζ0],
    label="nominal, OR7",
    marker=:circle, ms=3.0,
)
#plot!(cos.(angv), sin.(angv), label="Earth radius")
#plot!(b_E_2029()*cos.(angv), b_E_2029()*sin.(angv), label="Earth cross section")
plot!(
    R_Valsecchi_8_7*cos.(angv), 
    D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv), 
    label="8:7 res. Valsecchi circle",
    #legend=false,
)
plot!(
    R_Valsecchi_7_6*cos.(angv), 
    D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv), 
    label="7:6 res. Valsecchi circle",
)
plot!(
    R_Valsecchi_39_34*cos.(angv), 
    D_Valsecchi_39_34.+R_Valsecchi_39_34*sin.(angv), 
    label="39:34 res. Valsecchi circle",
)
plot!(
    R_Valsecchi_6_5*cos.(angv), 
    D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(angv), 
    label="6:5 res. Valsecchi circle",
    #aspectratio=0.8, #1
    #leg=false,
    legend=:outertopright,
    #legend=:topright,
    #legendfontsize=8
)
xlabel!("B.ξ [Earth radii]")
ylabel!("B.ζ [Earth radii]")
title!("B-plane (Öpik's frame)")
xlims!(1.47,1.51)
ylims!(6,9)

In [ ]:
x_OR7[7] # A_2 O+R fit

In [ ]:
sqrt(Γ_OR7[7,7]) # σ_{A_2} O+R fit

In [ ]:
# "Equivalent of Fig 1 in Farnocchia et al. (2013) Icarus, 224 192-200)"
# T = Tholen et al. (2013)
# R = radar (Giorgini 2008 + Brozovic 2018)
# T ellipsoid is expected to be roughly equivalent to Farnocchia's T ellipsoid,
# because dynamical models, astrometry and optical set and weights are the same; also A2 is assumed =0.
# T+R ellipsoid is not expected to be equivalent to Farnocchia's, since we include all radar astrometry through 2005-2013
# 7:6 and 8:7 Valsecchi circles, which enclose the ellipsoids, are out of the scale, so not shown

scatter(
    B_dot_ξv_O6*RE, 
    B_dot_ζv_O6*RE, 
    label="JTMC, O6 (A2=0)", 
    marker=:circle, ms=1.0, msc=:auto, # match marker stroke color
    markerstrokewidth = 0
)
scatter!(
    B_dot_ξv_OR6*RE, 
    B_dot_ζv_OR6*RE,
    label="JTMC, OR6 (A2=0)", 
    marker=:circle, ms=1.0, msc=:auto, # match marker stroke color
    markerstrokewidth = 0
)
scatter!(
    B_dot_ξv_OR7*RE, 
    B_dot_ζv_OR7*RE, 
    label="JTMC, OR7",  # A2=-0.82±2.8 10^{-14} au/day^2 (1-sigma)
    marker=:circle, ms=1.0, msc=:auto, # match marker stroke color
    markerstrokewidth = 0
)
plot!(
    B_ξ_3σ_ell_O6*RE,
    B_ζ_3σ_ell_O6*RE,
    label="3-σ, O6 (A2=0)",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR6*RE,
    B_ζ_3σ_ell_OR6*RE,
    label="3-σ, OR6 (A2=0)",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7*RE,
    B_ζ_3σ_ell_OR7*RE,
    label="3-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7_NEW*RE,
    B_ζ_3σ_ell_OR7_NEW*RE,
    label="3-σ, OR7_NEW",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!(
    [B_2029.ξ(x_O6)*RE],
    [B_2029.ζ(x_O6)*RE],
    label="nominal, O6 (A2=0)",
    marker=:circle,
)
scatter!(
    [B_2029.ξ(x_OR6)*RE],
    [B_2029.ζ(x_OR6)*RE],
    label="nominal, OR6 (A2=0)",
    marker=:circle,
)
scatter!(
    [B_2029.ξ(x_OR7)*RE],
    [B_2029.ζ(x_OR7)*RE],
    label="nominal, OR7",
    marker=:circle,
    #legend=false,
    legend=:outertopright,
    #legend=:bottomright
)
scatter!(
    [B_2029.ξ(x_OR7_NEW)*RE],
    [B_2029.ζ(x_OR7_NEW)*RE],
    label="nominal, OR7_NEW (A2=0)",
    marker=:circle,
)
#plot!(
#    RE*R_Valsecchi_7_6*cos.(angv),
#    RE*D_Valsecchi_7_6.+RE*R_Valsecchi_7_6*sin.(angv),
#    label="7:6 res. Valsecchi circle",
#)
#plot!(
#    RE*R_Valsecchi_8_7*cos.(angv),
#    RE*D_Valsecchi_8_7.+RE*R_Valsecchi_8_7*sin.(angv),
#    label="8:7 res. Valsecchi circle",
#)
xlabel!("B.ξ [km]")
ylabel!("B.ζ [km]")
#xlims!(9465,9520)
#ylims!(4.67e4, 4.86e4)
#yticks!(4.68e4:0.02e4:4.86e4)
#xticks!(9465:5:9520)

In [ ]:
#B_dot_ζv_OR7_NEW

In [ ]:
sqrt.(Γ_B_OR7_NEW[2,2])

In [ ]:
sqrt.(Γ_B_OR7[2,2])*RE

In [ ]:
B_2029.ξ(x_OR7_NEW), B_2029.ζ(x_OR7_NEW)

In [ ]:
gaussianpdf(x,μ=0.0,σ=1.0) = exp( -0.5((x-μ)/σ)^2 )/(σ*sqrt(2pi))

In [ ]:
B_2029.ζ(x_OR7)*RE

In [ ]:
B_2029.ζ(x_OR7_NEW)*RE

In [ ]:
#xv_plt_hst = collect(7.424:0.00001:7.4275)
xv_plt_hst = collect(7.2:0.00001:7.6)
yv_plt_hst_OR7 = gaussianpdf.(xv_plt_hst,B_2029.ζ(x_OR7),sqrt(Γ_B_OR7[2,2]))
yv_plt_hst_OR7_NEW = gaussianpdf.(xv_plt_hst,B_2029.ζ(x_OR7_NEW),sqrt(Γ_B_OR7_NEW[2,2]))
;

In [ ]:
#gr()
plotly()

In [ ]:
Plots.plot(xv_plt_hst*RE, yv_plt_hst_OR7/RE, label="OR7 (pre-2021)")
Plots.plot!(xv_plt_hst*RE, yv_plt_hst_OR7_NEW/RE, label="OR7 (post-2021)")
Plots.plot!([7.255*RE,7.255*RE], [0,10]/RE, label="2036 return")
Plots.plot!([7.734*RE,7.734*RE], [0,10]/RE, label="2068 return")
#Plots.histogram!(B_dot_ζv_OR7.*RE, normalize=:pdf)
#Plots.histogram!(B_dot_ζv_OR7_NEW.*RE, normalize=:pdf)
xlabel!("B.ζ  [km]")
ylabel!("Prob. density [km⁻¹]")
#xlims!(7.424,7.4275)
ylims!(0,10/RE)

In [ ]:
using Gaston

In [ ]:
Gaston.histogram(B_dot_ζv_OR7.*RE, normalize=:pdf,     Axes(
        linetype = :tab10,
        yrange=(0,15/RE),
        xlabel="'\$\\zeta_{2029}\$ [km]'",
        ylabel="'Prob. density [km\$^{-1}\$]'"
    )
)
Gaston.plot!(xv_plt_hst*RE, yv_plt_hst_OR7/RE, title="'OR7 (pre-2021)'", lw=3)
Gaston.plot!(xv_plt_hst*RE, yv_plt_hst_OR7_NEW/RE, title="'OR7 (post-2021)'", lw=3)
Gaston.plot!([7.249*RE,7.249*RE], [0,10]/RE, title="'2036 return'", lw=3)
Gaston.plot!([7.726*RE,7.726*RE], [0,10]/RE, title="'2068 return'", lw=3)


#Gaston.save(term = "cairolatex", output = "Bzeta_hist_OR7.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',14'")
#run(`pdflatex Bzeta_hist_OR7.tex`)

In [ ]:
7.249*RE,7.726*RE

In [ ]:
(7.249 - B_2029.ζ(x_OR7_NEW))/sqrt.(Γ_B_OR7_NEW[2,2]), (7.726 - B_2029.ζ(x_OR7_NEW))/sqrt.(Γ_B_OR7_NEW[2,2])

In [ ]:
gaussianpdf(7.249,B_2029.ζ(x_OR7_NEW),sqrt.(Γ_B_OR7_NEW[2,2])), gaussianpdf(7.726,B_2029.ζ(x_OR7_NEW),sqrt.(Γ_B_OR7_NEW[2,2]))

In [ ]:
gaussianpdf(7.249,B_2029.ζ(x_OR7),sqrt.(Γ_B_OR7[2,2])), gaussianpdf(7.726,B_2029.ζ(x_OR7),sqrt.(Γ_B_OR7[2,2]))

In [ ]:
#Gaston.histogram(B_dot_ζv_OR7.*RE, normalize=:pdf,     Axes(
#        linetype = :tab10,
#        yrange=(0,15/RE),
#        xlabel="'\$\\zeta_{2029}\$ [km]'",
#        ylabel="'Prob. density [km\$^{-1}\$]'"
#    )
#)
#Gaston.plot!(xv_plt_hst*RE, yv_plt_hst_OR7/RE, title="'OR7 (pre-2021)'", lw=3)

Gaston.plot(xv_plt_hst*RE, yv_plt_hst_OR7/RE, title="'OR7 (pre-2021)'", lw=4,
    Axes(
        key="top right box opaque spacing 1.4 width 1.4 height 0.4",
        linetype = :tab10,
        yrange=(0,15/RE),
        xlabel="'\$\\zeta_{2029}\$ [km]'",
        ylabel="'Prob. density [km\$^{-1}\$]'"
    )
)
Gaston.plot!(xv_plt_hst*RE, yv_plt_hst_OR7_NEW/RE, title="'OR7 (post-2021)'", lw=4)
Gaston.plot!([7.249*RE,7.249*RE], [0,20]/RE, title="'2036 return'", lw=4, dt=2)
Gaston.plot!([7.726*RE,7.726*RE], [0,20]/RE, title="'2068 return'", lw=4, dt=3)


#Gaston.save(term = "cairolatex", output = "Bzeta_hist_OR7.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',14'")
#run(`pdflatex Bzeta_hist_OR7.tex`)

In [ ]:
x0_JPL = vcat(1e8ones(6),1.0).*(vcat(Apophis.x0_JPL_s199, A2_JPL) .- vcat(Apophis.x0_JPL_s197, 0.0))

In [ ]:
julian2datetime(jd0+t_2029(x0_JPL))

In [ ]:
julian2datetime.(jd0 .+ tvS1(vcat(zeros(6), 0.0)))[end-30:end]

In [ ]:
t_2020 = tvS1[27]
t_2021 = tvS1[29]
t_2029 = tvS1[47];

In [ ]:
julian2datetime(jd0+t_2020(x0_JPL))

In [ ]:
julian2datetime(jd0+t_2021(x0_JPL))

In [ ]:
julian2datetime(jd0+t_2029(x0_JPL))

In [ ]:
ss16asteph_2020 = ss16asteph((jd0-J2000+t_2020)*daysec)
xe_2020 = ss16asteph_2020[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

ss16asteph_2021 = ss16asteph((jd0-J2000+t_2021)*daysec)
xe_2021 = ss16asteph_2021[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

ss16asteph_2029 = ss16asteph((jd0-J2000+t_2029)*daysec)
xe_2029 = ss16asteph_2029[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

In [ ]:
apophis_2020 = xvS1[27,1:6]
apophis_2021 = xvS1[29,1:6]
apophis_2029 = xvS1[47,1:6]
xae_2020 = apophis_2020-xe_2020
xae_2021 = apophis_2021-xe_2021
xae_2029 = apophis_2029-xe_2029;

In [ ]:
rae_2020 = sqrt(xae_2020[1]^2 + xae_2020[2]^2 + xae_2020[3]^2);

In [ ]:
rae_2021 = sqrt(xae_2021[1]^2 + xae_2021[2]^2 + xae_2021[3]^2);

In [ ]:
rae_2029 = sqrt(xae_2029[1]^2 + xae_2029[2]^2 + xae_2029[3]^2);

In [ ]:
au*(rae_2020(x_OR7) - 0.216276663586499)

In [ ]:
au*(rae_2021(x_OR7) - 0.11265113489653)

In [ ]:
au*(rae_2029(x_OR7) - 0.000252172816142565)

In [ ]:
julian2datetime(2462240.407032288)

In [ ]:
#
0.21593875724774*60/3

In [ ]:
0.000252172816142565*au

In [ ]:
d_CA_2029_JPL = 0.000252172816142565
d_CA_2029_JPL_min = 0.000247515097583323
d_CA_2029_JPL_max = 0.000256837212164523

In [ ]:
(d_CA_2029_JPL-d_CA_2029_JPL_min)*au/3, (d_CA_2029_JPL-d_CA_2029_JPL_max)*au/3

In [ ]:
julian2datetime(jd0+t_2029(x_OR6))

In [ ]:
julian2datetime(jd0+t_2029(x_OR7))

In [ ]:
au*rae_2029(x_OR7)

In [ ]:
au*rae_2029(x_OR6)

In [ ]:
au*(rae_2020(x0_JPL) - 0.216276663586499)

In [ ]:
au*(rae_2021(x0_JPL) - 0.11265113489653)

In [ ]:
au*(rae_2029(x0_JPL) - 0.000252172816142565)

In [ ]:
#t_2020

In [ ]:
#rae_2020

In [ ]:
#t_CA = t_2020
#d_CA = rae_2020

t_CA = t_2029
d_CA = rae_2029;

In [ ]:
# Jacobian of transformation from initial conditions to d_CA_2020, t_CA_2020
t_x2dtCA = hcat(TaylorSeries.gradient(d_CA), TaylorSeries.gradient(t_CA));
size(t_x2dtCA)

In [ ]:
# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_dtCA_OR7 = Hermitian(t_x2dtCA(x_OR7)'*Γ_OR7*(t_x2dtCA(x_OR7)))
Γ_dtCA_OR7_NEW = Hermitian(t_x2dtCA(x_OR7_NEW)'*Γ_OR7_NEW*(t_x2dtCA(x_OR7_NEW)))
Γ_dtCA_OR6 = Hermitian(t_x2dtCA(x_OR6)[1:6,:]'*Γ_OR6*(t_x2dtCA(x_OR6)[1:6,:]))
# get cholesky-L factor
L_dtCA_OR7 = cholesky(Γ_dtCA_OR7).L
L_dtCA_OR6 = cholesky(Γ_dtCA_OR6).L

In [ ]:
sqrt.(diag(Γ_dtCA_OR6)).*[au,daysec]

In [ ]:
sqrt.(diag(Γ_dtCA_OR7)).*[au,daysec]

In [ ]:
sqrt.(diag(Γ_dtCA_OR7_NEW)).*[au,daysec]

In [ ]:
dt_v_OR6 = Matrix{Float64}(undef, 1000, 2)
dt_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    dt_v_OR6[i,:] = L_dtCA_OR6*randn(2)
    dt_v_OR7[i,:] = L_dtCA_OR7*randn(2)
end

In [ ]:
d_3σ_ell_OR6 = (L_dtCA_OR6[1,1]*rndv_2sph_ξ .+ L_dtCA_OR6[1,2]*rndv_2sph_ζ) # .+ d_CA(x_OR6)
t_3σ_ell_OR6 = (L_dtCA_OR6[2,1]*rndv_2sph_ξ .+ L_dtCA_OR6[2,2]*rndv_2sph_ζ) # .+ t_CA(x_OR6)

d_3σ_ell_OR7 = (L_dtCA_OR7[1,1]*rndv_2sph_ξ .+ L_dtCA_OR7[1,2]*rndv_2sph_ζ) # .+ d_CA(x_OR7)
t_3σ_ell_OR7 = (L_dtCA_OR7[2,1]*rndv_2sph_ξ .+ L_dtCA_OR7[2,2]*rndv_2sph_ζ) # .+ t_CA(x_OR7)

Δd_OR76 = d_CA(x_OR7)-d_CA(x_OR6)
Δt_OR76 = t_CA(x_OR7)-t_CA(x_OR6)
;

In [ ]:
plotly()

In [ ]:
scatter(
    (Δd_OR76.+dt_v_OR7[:,1])*au, 
    (Δt_OR76.+dt_v_OR7[:,2])*daysec, 
    ms=0.1, msc=:auto, label="OR7 (JT sampling)",
    legend=:outertopright
)
scatter!(dt_v_OR6[:,1]*au, dt_v_OR6[:,2]*daysec, ms=0.1, msc=:auto, label="OR6 (JT sampling)")
plot!(d_3σ_ell_OR6*au, t_3σ_ell_OR6*daysec, label="OR6 (3-σ)")
plot!((Δd_OR76.+d_3σ_ell_OR7)*au, (Δt_OR76.+t_3σ_ell_OR7)*daysec, label="OR7 (3-σ)")
scatter!([Δd_OR76*au], [Δt_OR76*daysec], label="OR7, nominal")
xlabel!("Geocentric range at closest approach [km]")
ylabel!("Time of closest approach [s]")
yca = year(julian2datetime(jd0 + t_CA(x_OR7)))
mca = monthname(julian2datetime(jd0 + t_CA(x_OR7)))
title!("$mca $yca close approach: CA time and distance")

In [ ]:
d_CA(x_OR6), julian2datetime(jd0 + t_CA(x_OR6))

In [ ]:
au*d_CA(x_OR6)

In [ ]:
d_CA(x_OR7), julian2datetime(jd0 + t_CA(x_OR7))

In [ ]:
au*0.000252172816142565

In [ ]:
au*d_CA(x_OR7)

In [ ]:
d_CA(x_OR7_NEW), julian2datetime(jd0 + t_CA(x_OR7_NEW))

In [ ]:
au*d_CA(x_OR7_NEW)

In [ ]:
sqrt(Γ_dtCA_OR7_NEW[1,1])*au

# 2020-2021 RA/DEC OR6/7 prediction vs 2020 observation

In [ ]:
using JuliaDB

In [ ]:
t_radec_1120 = JuliaDB.load("radec_99942_2011_2020.jdb");

In [ ]:
t_radec_1120[end].dt_utc_obs

In [ ]:
t_radec_1120[end].obscode # G96 = Mt. Lemmon

In [ ]:
t_radec_1120[end].α_comp()

In [ ]:
ra_last_2020_mpc = (t_radec_1120[end].α_obs - t_radec_1120[end].α_corr) - t_radec_1120[end].α_comp 
dec_last_2020_mpc = (t_radec_1120[end].δ_obs - t_radec_1120[end].δ_corr) - t_radec_1120[end].δ_comp;

In [ ]:
σ_level_plot = 3.0

In [ ]:
yTN = dec_last_2020_mpc
xTN = ra_last_2020_mpc

ra_res_nom_OR6 = xTN(x_OR6)/3600
dec_res_nom_OR6 = yTN(x_OR6)/3600
ra_res_nom_OR7 = xTN(x_OR6)/3600
dec_res_nom_OR7 = yTN(x_OR6)/3600

@show ra_res_nom_OR6, dec_res_nom_OR6
@show ra_res_nom_OR7, dec_res_nom_OR7

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = σ_level_plot*cos.(angv)
rndv_2sph_yTN = σ_level_plot*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

#plotly()
scatter( ΔxTN_OR76.+xyTN_v_OR7[:,1], ΔyTN_OR76.+xyTN_v_OR7[:,2], ms=0.1, msc=:auto, label="OR7 (JT sampling)")
scatter!(xyTN_v_OR6[:,1], xyTN_v_OR6[:,2], ms=0.1, msc=:auto, label="OR6 (JT sampling)", legend=:outertopright)
#scatter([0], [0], label="OR6 nominal")
scatter!([ΔxTN_OR76], [ΔyTN_OR76], label="OR7 nominal")
plot!(xTN_3σ_ell_OR6, yTN_3σ_ell_OR6, label="OR6 (3-σ)")
plot!(ΔxTN_OR76.+xTN_3σ_ell_OR7, ΔyTN_OR76.+yTN_3σ_ell_OR7, label="OR7 (3-σ)")
xlabel!("RA*cos(DEC) [arcsec]")
ylabel!("DEC [arcsec]")
#yca = year(julian2datetime(jd0 + t_2020(x_OR7)))
#mca = monthname(julian2datetime(jd0 + t_2020(x_OR7)))
title!("RA/DEC observations predictions \n (MPC obs. code 568)")

In [ ]:
### https://newton.spacedys.com/neodys/index.php?n=99942&pc=1.1.4.1&oc=G96&y0=2020&m0=06&d0=08&h0=04&mi0=22&s=3.0&xf=0.0&yf=0.0
### Astrometric Observation Prediction
### For 2020/06/08 04:22:00 (UTC);  59008.18194(MJD)
### Observatory code = 1696
### RA = 09:35:13.486 (HH:MM:SS);   143.80619 (deg)
### DEC = +14 38 44.24 (deg min sec);    14.64562 (deg)
### RA*cos(DEC)/DEC Apparent motion =      1.832     -0.466 (arcsec/min)        30.5        -7.8 (marcsec/s)
### Apparent motion velocity =     1.890 arcsec/min) and Position Angle = 104.258 (deg)
### Sun distance =   0.9912 (au)
### Earth distance =   0.8405 (au)
### Sun elevation above horizon =  -20.04 (deg)
### Solar elongation =  -63.77 (deg)    Lunar elongation =  147.67 (deg)
### Galactic latitude =   42.81 (deg)   Galactic longitude =  217.94 (deg)
### Apparent magnitude = 20.85
### Phase angle =   66.72 (deg)
### Altitude =   26.46 (deg)    Airmass =   2.232
### Size and orientation of 1-sigma uncertainty ellipse
### Short axis : Size = 7.468853E-04 (arcsec); Direction =   0.99653 -0.08329
### Long axis : Size = 6.657841E-03 (arcsec); Direction =   0.08329  0.99653
### The confidence boundary shown below has been computed using the linear theory.

In [ ]:
σ_level_plot = 1.0

In [ ]:
angs = range(0.0, stop=2.01pi, length=100)
cos_angs = cos.(angs)
sin_angs = sin.(angs)

a_ = 6.657841E-03 # long axis size (arcsec)
b_ = 7.468853E-04 # short axis size (arcsec)
cos_ = 0.08329 # long axis u_x
sin_ = 0.99653 # long axis u_y

x_neodys = (a_*cos_).*cos_angs .- (b_*sin_).*sin_angs
y_neodys = (a_*sin_).*cos_angs .+ (b_*cos_).*sin_angs

radec_xσ_neodys = σ_level_plot .* [x_neodys y_neodys];

In [ ]:
#plotly()
scatter(xyTN_v_OR6[:,1], xyTN_v_OR6[:,2], ms=0.1, msc=:auto, label="OR6 (MC sampling)", leg=:bottomright)
#scatter([0], [0], label="OR6 nominal")
scatter!([ra_res_nom_OR6], [dec_res_nom_OR6], label="Res. (O-C, OR6 nominal)", m=:diamond, ms=8)
plot!((σ_level_plot/3).*xTN_3σ_ell_OR6, (σ_level_plot/3).*yTN_3σ_ell_OR6, label="OR6 ($(Int(σ_level_plot))-σ)", lw=2)
plot!(xTN_3σ_ell_OR6, yTN_3σ_ell_OR6, label="OR6 (3-σ)", lw=2)
plot!(3radec_xσ_neodys[:,1], 3radec_xσ_neodys[:,2], label="NEODyS ($(Int(3σ_level_plot))-σ)", lw=2)

xlabel!("RA*cos(DEC) [arcsec]")
ylabel!("DEC [arcsec]")
#yca = year(julian2datetime(jd0 + t_2020(x_OR7)))
#mca = monthname(julian2datetime(jd0 + t_2020(x_OR7)))
title!("RA/DEC obs. residual: Mt. Lemmon (G96) \n Obs. epoch: 2020-06-08 04:22 UTC")
xlims!(-0.003,0.004)
ylims!(-0.025,0.025)
#xticks!(-0.1:0.02:0.1)
#yticks!(-0.14:0.02:0.14)

# Predicted radar observations: 2021 at Goldstone

In [ ]:
file_dict = JLD.load("deldop_99942_2021_predobs_Goldstone_o5v7.jld")

In [ ]:
Apophis.loadjpleph()

In [ ]:
Apophis.delay_doppler2(253, DateTime(2021,3,6), 2380.0, xva=Apophis.apophis_pv_199, lod=false, eocorr=false)

In [ ]:
xTN = file_dict["vdel"][1]
yTN = file_dict["vdop"][1]

@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

#plotly()
scatter( ΔxTN_OR76.+xyTN_v_OR7[:,1], ΔyTN_OR76.+xyTN_v_OR7[:,2], ms=0.1, msc=:auto, label="OR7 (JT sampling)")
scatter!(xyTN_v_OR6[:,1], xyTN_v_OR6[:,2], ms=0.1, msc=:auto, label="OR6 (JT sampling)")
#scatter([0], [0], label="OR6 nominal")
scatter!([ΔxTN_OR76], [ΔyTN_OR76], label="OR7 nominal")
plot!(xTN_3σ_ell_OR6, yTN_3σ_ell_OR6, label="OR6 (3-σ)")
plot!(ΔxTN_OR76.+xTN_3σ_ell_OR7, ΔyTN_OR76.+yTN_3σ_ell_OR7, label="OR7 (3-σ)")
xlabel!("Delay [us]")
ylabel!("Doppler [Hz]")
#yca = year(julian2datetime(jd0 + t_2020(x_OR7)))
#mca = monthname(julian2datetime(jd0 + t_2020(x_OR7)))
title!("Goldstone March 2021 radar observations predictions")

# Cartesian coordinates to Keplerian elements transformation

In [ ]:
ss16asteph_0 = ss16asteph((jd0-J2000)*daysec)
xs0 = ss16asteph_0[union(3su-2:3su,3(N-1+su)-2:3(N-1+su))]

In [ ]:
Apophis.initialcond(dq)

In [ ]:
ϵ0_deg = 84381.448/3600

In [ ]:
m_eq2ecl = Rx(deg2rad(ϵ0_deg))

In [ ]:
m_xv_eq2ecl = hcat(vcat(m_eq2ecl, zeros(3,3)), vcat(zeros(3,3), m_eq2ecl))

In [ ]:
q0_197 = zeros(6)
q0_197[1:3] .= [-0.9633018953468989, 0.5100289806011301, 0.16528024397505386]
q0_197[4:6] .= [-0.0071188720570829036, -0.012061235689040535, -0.0046695143453363164]
q0_197

In [ ]:
#xas0 = m_xv_eq2ecl*( q0_197 + (jt_scaling.*dq)[1:6] - xs0 ) # taking S197 initial conditions, useful to check vs JPL elements
xas0 = m_xv_eq2ecl*(  Apophis.initialcond(jt_scaling.*dq)[1:6] - xs0  )

In [ ]:
# Solution #197 elements

EPOCH_JPL197 = 2454733.5
EC_JPL197 = .1911952942528226;  QR_JPL197 = .7460724385331012;  TP_JPL197 = 2454894.912507658200
OM_JPL197 = 204.4460284242489;  W_JPL197 = 126.401880836064;  IN_JPL197 = 3.331369228495799
kep_JPL197 = [EC_JPL197, QR_JPL197, TP_JPL197, OM_JPL197, W_JPL197, IN_JPL197]

In [ ]:
ec0 = eccentricity(xas0..., μ[su], 0.0)
a0 = semimajoraxis(xas0..., μ[su], 0.0)
qr0 = a0*(1-ec0)
tp0 = timeperipass(2.4547335e6, xas0..., μ[su], 0.0)
Ω0 = rad2deg(longascnode(xas0...))
ω0 = rad2deg(argperi(xas0..., μ[su], 0.0))
i0 = rad2deg(inclination(xas0...))
A20 = jt_scaling[7]*dq[7]
kep_OR6 = [ec0(x_OR6), qr0(x_OR6), tp0(x_OR6), Ω0(x_OR6), ω0(x_OR6), i0(x_OR6), A20(x_OR6)]
kep_OR7 = [ec0(x_OR7), qr0(x_OR7), tp0(x_OR7), Ω0(x_OR7), ω0(x_OR7), i0(x_OR7), A20(x_OR7)]
kep_197 = [ec0(), qr0(), tp0(), Ω0(), ω0(), i0(), A20()]

In [ ]:
(kep_JPL197 .- kep_OR6[1:6])

In [ ]:
(kep_JPL197 .- kep_197[1:6])./eps.(kep_JPL197)

In [ ]:
kep0 = [ec0, qr0, tp0, Ω0, ω0, i0, A20];

In [ ]:
TaylorSeries.jacobian(kep0)'

In [ ]:
t_car2kep(x) = TaylorSeries.jacobian(kep0, x)'
t_car2kep() = TaylorSeries.jacobian(kep0)'

In [ ]:
t_car2kep()

In [ ]:
t_car2kep() .- TaylorSeries.jacobian(kep0)'

In [ ]:
Γ_O7_kep = (t_car2kep(x_O7)')*Γ_O7*(t_car2kep(x_O7));
Γ_O6_kep = (t_car2kep(x_O6)[1:6,1:6]')*Γ_O6*t_car2kep(x_O6)[1:6,1:6];
Γ_OR7_kep = (t_car2kep(x_OR7)')*Γ_OR7*(t_car2kep(x_OR7));
Γ_OR6_kep = (t_car2kep(x_OR6)[1:6,1:6]')*Γ_OR6*t_car2kep(x_OR6)[1:6,1:6];

In [ ]:
for (x, Γ_kep) in [(x_O6, Γ_O6_kep), (x_OR6, Γ_OR6_kep), (x_O7, Γ_O7_kep), (x_OR7, Γ_OR7_kep)]
    @show kep0(x) sqrt.(diag(Γ_kep))
    println("* ----===OOO===--- *")
end

In [ ]:
kep0(x_OR7)

In [ ]:
sqrt(1/(inv(Γ_OR7)[7,7]))

In [ ]:
sqrt(1/(inv(Γ_OR7_kep)[7,7]))

In [ ]:
sqrt.(diag(Γ_OR7_kep))

In [ ]:
kep0(x_OR6)

In [ ]:
sqrt.(diag(Γ_OR6_kep))

In [ ]:
Γ_kep_JPL199 = [ 2.858062040774483E-17 -2.531903029559859E-17   2.451271598753632E-14     5.341143529000106E-14     -5.064059304794175E-14     -2.316088496620918E-16     -9.159706240530915E-23;
-2.531903029559859E-17 2.250437378457998E-17   -2.122266051203493E-14     -5.100944859184798E-14     4.826834522209508E-14     2.76636664162405E-16     7.744081722922498E-23;
 2.451271598753632E-14 -2.122266051203493E-14   2.501437350446983E-11     1.16213242311946E-11     -1.057653022558682E-11     4.597741246907787E-13     -1.026132427113056E-19;
 5.341143529000106E-14 -5.100944859184798E-14   1.16213242311946E-11     4.437388022822618E-10     -4.338840441053813E-10     -6.817498150073878E-12     -4.324000765783613E-21;
-5.064059304794175E-14 4.826834522209508E-14   -1.057653022558682E-11     -4.338840441053813E-10     4.261149273549589E-10     6.697032883689781E-12     1.139615135464918E-20;
-2.316088496620918E-16 2.76636664162405E-16   4.597741246907787E-13     -6.817498150073878E-12     6.697032883689781E-12     1.22672053287042E-13     -2.524941685627351E-21;
-9.159706240530915E-23 7.744081722922498E-23   -1.026132427113056E-19     -4.324000765783613E-21     1.139615135464918E-20     -2.524941685627351E-21     4.846398125111792E-28]


In [ ]:
Γ_kep_JPL201 = [ 1.047666119400409E-17 -9.926055175063387E-18  4.909881116021864E-15  4.653843296950616E-14 -4.259843629461694E-14 -6.166837591066471E-16  7.335166335864171E-26
-9.926055175063387E-18  9.410682503015075E-18 -4.591352679580688E-15 -4.484904345608430E-14  4.112408300414402E-14  5.980282716461667E-16 -2.967424019160920E-25
 4.909881116021864E-15 -4.591352679580688E-15  3.593909565851593E-12  5.892040879932584E-12 -3.541800585772594E-12  1.014282699780642E-14 -2.345777043520376E-21
 4.653843296950616E-14 -4.484904345608430E-14  5.892040879932584E-12  4.259262167985737E-10 -4.159320593828374E-10 -6.673698596512044E-12  2.186907592906685E-20
-4.259843629461694E-14  4.112408300414402E-14 -3.541800585772594E-12 -4.159320593828374E-10  4.079345990507304E-10  6.583991773293156E-12 -2.094895343691202E-20
-6.166837591066471E-16  5.980282716461667E-16  1.014282699780642E-14 -6.673698596512044E-12  6.583991773293156E-12  1.089504866028434E-13 -4.103375903026154E-22
 7.335166335864171E-26 -2.967424019160920E-25 -2.345777043520376E-21  2.186907592906685E-20 -2.094895343691202E-20 -4.103375903026154E-22  1.511466172500938E-29]

In [ ]:
sqrt.(diag(Γ_kep_JPL201))

In [ ]:
sqrt(inv(inv(Γ_kep_JPL201)[7,7]))

In [ ]:
Γ_OR7_kep

In [ ]:
(Γ_kep_JPL199 .- Γ_OR7_kep)./Γ_kep_JPL199

In [ ]:
Γ_OR7_kep ./ Γ_kep_JPL199

In [ ]:
kep_OR7

In [ ]:
orbel_OR7_TN = kep_OR7 .+ dq

In [ ]:
ady_kep_au_My = 1e6yr*yarkp2adot(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) # au/Myr

In [ ]:
ady_kep_m_y = 1e3au*yr*yarkp2adot(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) #m/yr

In [ ]:
t_kep2ady(x, ady_kep) = TaylorSeries.gradient(ady_kep)(x)

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_au_My)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_au_My) )

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_m_y)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_m_y) )

$(-23 \pm 11)\times 10^{-4}$au/Myr


$(-341 \pm 158)$m/yr

# Gnuplot

In [ ]:
using Gnuplot

## 2029 B-plane, zoom out

In [ ]:
nomps = 0.08

@gp "unset grid" :-
#@gp :- "set si sq" :-
@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :- # inside box" :-
#@gp :- xrange=[1.48, 1.50] :-
#@gp :- yrange=[7.1, 7.6] :-
@gp :- "set xlabel '\$\\xi\$ [\$R_\\oplus\$]'" :-
@gp :- "set ylabel '\$\\zeta\$ [\$R_\\oplus\$]'" :-

@gp :- "set style fill transparent solid 0.35 noborder" :-
@gp :- "set style circle radius $nomps" :-

#@gp :-B_dot_ξv_O6*au/RE B_dot_ζv_O6*au/RE "pt 7 ps 0.2 lt 4 t 'JT O6'" :-
#@gp :- B_ξ_3σ_ell_O6 B_ζ_3σ_ell_O6 "w l lw 3 lt 4 t 'O6 3-\$\\sigma\$'" :-
#@gp :- B_dot_ξ(x_O6)*au/RE B_dot_ζ(x_O6)*au/RE "pt 7 ps 0.75 lt 4  t 'O6 nominal'" :-

#@gp :- B_dot_ξv_O7*au/RE B_dot_ζv_O7*au/RE "pt 7 ps 0.2 lt 1 t 'JT O7'" :-
#@gp :- B_ξ_3σ_ell_O7 B_ζ_3σ_ell_O7 "w l lw 3 lt 1 t 'O7 3-\$\\sigma\$'" :-
#@gp :- B_dot_ξ(x_O7)*au/RE B_dot_ζ(x_O7)*au/RE "pt 7 ps 0.75 lt 1  t 'O7 nominal'" :-

#@gp :- B_dot_ξv_OR6*au/RE B_dot_ζv_OR6*au/RE "pt 7 ps 0.2 lt 3 t 'JT OR6'" :-
#@gp :- B_ξ_3σ_ell_OR6 B_ζ_3σ_ell_OR6 "w l lw 3 lt 3 t 'OR6 3-\$\\sigma\$'" :-
#@gp :- B_dot_ξ(x_OR6)*au/RE B_dot_ζ(x_OR6)*au/RE "pt 7 ps 0.75 lt 3 t 'OR6 nominal'" :-

#@gp :- B_dot_ξv_OR7*au/RE B_dot_ζv_OR7*au/RE "pt 7 ps 0.2 lt 2 t 'JT OR7'" :-

#@gp :- cos.(angv) sin.(angv) "w l lt 6 dashtype 2 t'Earth radius'" :-
@gp :- B_2029.ξ(x_OR7) B_2029.ζ(x_OR7) "w circle lc rgb 'white' fs solid 1.0 border lt -1 t 'OR7 nominal'" :-
@gp :- B_ξ_5σ_ell_OR7 B_ζ_5σ_ell_OR7 "w l lw 2 lc rgb 'gray' t 'OR7 5-\$\\sigma\$'" :-
@gp :- B_ξ_3σ_ell_OR7 B_ζ_3σ_ell_OR7 "w l lw 2 lt 2 lc rgb 'black' t 'OR7 3-\$\\sigma\$'" :-

@gp :- B_2029.b()*cos.(angv) B_2029.b()*sin.(angv) "w l lw 2 lc rgb 'black' dashtype 1 t'Earth cross section'" :-
#@gp :- R_Valsecchi_6_5*cos.(angv) (D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(angv)) "w l lt 3 t'Valsecchi circle 6:5'" :-
@gp :- R_Valsecchi_8_7*cos.(angv) (D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv)) "w l lw 2 lc rgb 'black' dashtype 3 t 'Valsecchi circle 8:7'" :-
@gp :- R_Valsecchi_7_6*cos.(angv) (D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv)) "w l lw 2 lc rgb 'black' dashtype 6 t 'Valsecchi circle 7:6'"

#Gnuplot.save(term="epslatex standalone color dashed size 7in,6in font \",12\"", output="bplane_2029_RE__.tex")
#run(`pdflatex bplane_2029_RE__.tex`)

## 2029 B-plane, zoom IN

In [ ]:
jtps = 0.5
ellw = 5.0
nomps = 1.0
angv_vc = (pi/4):0.001:(pi/2);

@gp "unset grid" :-
#@gp "set si sq" :-
#@gp :- "set size ratio 1.2" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[9460, 9485] :-
@gp :- xrange=[9450, 9490] :-
#@gp :- yrange=[45000, 49000] :-
#@gp :- yrange=[46200, 48100] :-
@gp :- "set xlabel '\$\\xi\$ [km]'" :-
@gp :- "set ylabel '\$\\zeta\$ [km]'" :-

@gp :- "set xtics 10" :-
@gp :- "set ytics 400" :-

@gp :- "set style fill transparent solid 0.35 noborder" :-
@gp :- "set style circle radius $nomps" :-

#@gp :- B_dot_ξv_O7*RE B_dot_ζv_O7*RE "pt 7 ps $jtps lt 4 t 'O7 (MC sampling)'" :-
#@gp :- B_dot_ξv_O6*RE B_dot_ζv_O6*RE "pt 7 ps $jtps lt 1 t 'O6 (MC sampling)'" :-
#@gp :- B_dot_ξv_OR6*RE B_dot_ζv_OR6*RE "pt 7 ps $jtps lt 3 t 'OR6 (MC sampling)'" :-
#@gp :- B_dot_ξv_OR7*RE B_dot_ζv_OR7*RE "pt 7 ps $jtps lc rgb 'black' t 'OR7 (MC sampling)'" :-
#@gp :- B_dot_ξv_OR7_NEW*RE B_dot_ζv_OR7_NEW*RE "pt 7 ps $jtps lc 4 t 'OR7 (MC sampling)'" :-

#@gp :- B_ξ_3σ_ell_O6*RE B_ζ_3σ_ell_O6*RE "w l lw $ellw lt 1 t 'O6 (3-\$\\sigma\$)'" :-
#@gp :- B_ξ_3σ_ell_O7*RE B_ζ_3σ_ell_O7*RE "w l lw $ellw lt 4 t 'O7 (3-\$\\sigma\$)'" :-
@gp :- B_ξ_3σ_ell_OR6*RE B_ζ_3σ_ell_OR6*RE "w l lw $ellw lt 3 t 'OR6 (3-\$\\sigma\$)'" :-
@gp :- B_ξ_3σ_ell_OR7*RE B_ζ_3σ_ell_OR7*RE "w l lw $ellw lt 2 lc rgb 'black' t 'OR7 (3-\$\\sigma\$)'" :-
@gp :- B_ξ_3σ_ell_OR7_NEW*RE B_ζ_3σ_ell_OR7_NEW*RE "w l lw $ellw lt 4 lc 4 t 'OR7 + 2021 data (3-\$\\sigma\$)'" :-
#@gp :- B_ξ_5σ_ell_OR7*RE B_ζ_5σ_ell_OR7*RE "w l lw $ellw lc rgb 'gray' t 'OR7 (5-\$\\sigma\$)'" :-

#@gp :- B_2029.ξ(x_O6)*RE B_2029.ζ(x_O6)*RE "w circle lt 1 fs solid 1.0 border lt -1 t 'O6 nominal'" :-
#@gp :- B_2029.ξ(x_O7)*RE B_2029.ζ(x_O7)*RE "w circle lt 4 fs solid 1.0 border lt -1  t 'O7 nominal'" :-
@gp :- B_2029.ξ(x_OR6)*RE B_2029.ζ(x_OR6)*RE "w circle lt 3 fs solid 1.0 border lt -1 t 'OR6 (nominal)'" :-
@gp :- B_2029.ξ(x_OR7)*RE B_2029.ζ(x_OR7)*RE "w circle lc rgb 'gray' fs solid 1.0 border lt -1 t 'OR7 (nominal)'" :-
@gp :- B_2029.ξ(x_OR7_NEW)*RE B_2029.ζ(x_OR7_NEW)*RE "w circle lc 4 fs solid 1.0 border lt -1 t 'OR7 + 2021 data (nominal)'" :-

#@gp :- RE*R_Valsecchi_8_7*cos.(angv_vc) RE*(D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv_vc)) "w l lw 2 lc rgb 'black' dashtype 3 t 'Valsecchi circle 8:7'" :-
@gp :- RE*R_Valsecchi_7_6*cos.(angv_vc) RE*(D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv_vc)) "w l lw 2 lc rgb 'black' dashtype 6 t '2036 Valsecchi circle'"
@gp :- RE*R_Valsecchi_39_34*cos.(angv_vc) RE*(D_Valsecchi_39_34.+R_Valsecchi_39_34*sin.(angv_vc)) "w l lw 2 lc rgb 'black' dashtype 4 t '2068 Valsecchi circle'"

#@gp :- cos.(angv) sin.(angv) "w l lt 6 dashtype 2 t'Earth radius'" :-
#@gp :- b_E_2029()*cos.(angv) b_E_2029()*sin.(angv) "w l lw 2 lc rgb 'black' dashtype 6 t'Earth cross section'"

#Gnuplot.save(term="epslatex standalone color dashed size 8in,6in font \",12\"", output="bplane_2029_RE_zoom_PDC2021.tex")
#run(`pdflatex bplane_2029_RE_zoom_PDC2021.tex`)

## Mar 2021 RA/DEC obs prediction, relative to nominal OR6 orbit

In [ ]:
Gnuplot.options.gpviewer = true

jtps = 0.25
ellw = 4.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

yTN = dec_last_2020_mpc
xTN = ra_last_2020_mpc

@show xTN(x_OR6)/3600, yTN(x_OR6)/3600
@show xTN(x_OR7)/3600, yTN(x_OR7)/3600

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

@gp "set multiplot"
#@gp :- "set title 'Mt. Lemmon (G96), epoch: 2020-06-08 04:22 UTC'" :-
@gp :- "unset grid" :-
@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[-0.2,0.06] :-
@gp :- yrange=[-0.02,0.07] :-
#@gp :- "set xlabel '\$\\alpha\\cos(\\delta)-\\alpha_\\text{OR6}\\cos(\\delta_\\text{OR6})\$ [arcsec]'" :-
#@gp :- "set ylabel '\$\\delta-\\delta_\\text{OR6}\$ [arcsec]'" :-
@gp :- "set xlabel '\$\\alpha\\cos(\\delta)\$ [arcsec]'" :-
@gp :- "set ylabel '\$\\delta\$ [arcsec]'" :-


#@gp :- "set xtics 20" :-

#@gp :- "set style fill transparent solid 0.35 noborder" :-
#@gp :- "set style circle radius $nomps" :-

@gp :- 1 ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 t '7DOF (MC sampling)'" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 t '6DOF (MC sampling)'" :-

@gp :- ΔxTN_OR76.+ xTN_3σ_ell_OR7 ΔyTN_OR76.+ yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' t '7DOF (3-\$\\sigma\$)'" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 t '6DOF (3-\$\\sigma\$)'" :-

@gp :- 3radec_xσ_neodys[:,1] 3radec_xσ_neodys[:,2] "w l lw $ellw lt 4 t 'NEODyS (3-\$\\sigma\$)'" :-

@gp :- ΔxTN_OR76 ΔyTN_OR76 "w circle lc rgb 'white' fs solid 1.0 border lt -1 t '7DOF (nominal)'" :-

@gp :- [ra_res_nom_OR6] [dec_res_nom_OR6] "pt 1 ps 3 lw 4 lc rgb 'red' t 'observed'" :-

@gp :- 2 "set origin 0.2125, 0.42" :-
@gp :- "set size 0.5, 0.6" :-
@gp :- xrange=[-0.003, 0.003] :-
@gp :- yrange=[-0.022, 0.022] :-
@gp :- "unset xlabel" :-
@gp :- "unset ylabel" :-
#@gp :- "unset title" :-
@gp :- "set xtics ('-3\$\\cdot 10^{-3}\$' -0.003, \"0\" 0, '3\$\\cdot 10^{-3}\$' 0.003)" :-
@gp :- "set ytics ('-1.75\$\\cdot 10^{-2}\$' -0.0175, \"0\" 0, '1.75\$\\cdot 10^{-2}\$' 0.0175)" :-

@gp :- ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 not" :-
@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- 3radec_xσ_neodys[:,1] 3radec_xσ_neodys[:,2] "w l lw $ellw lt 4 not" :-
@gp :- [ra_res_nom_OR6] [dec_res_nom_OR6] "pt 1 ps 3 lw 4 lc rgb 'red' not"
#@gp :- "unset multiplot"

#Gnuplot.save(term="epslatex standalone color dashed size 7in,5in font \",11\"", output="radec_2021.tex")
#run(`pdflatex radec_2021.tex`)

Gnuplot.options.gpviewer = false;

## March 2021 Goldstone radar (delay/Doppler) prediction, relative to nominal OR6 orbit

In [ ]:
Gnuplot.options.gpviewer = true

jtps = 0.25
ellw = 3.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

xTN = file_dict["vdel"][1]
yTN = file_dict["vdop"][1]

@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv)

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN)

#xTN_2p5σ_ell_OR7 = (2.5/3)*xTN_3σ_ell_OR7
#yTN_2p5σ_ell_OR7 = (2.5/3)*yTN_3σ_ell_OR7

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

@gp "set multiplot"
@gp :- "unset grid" :-
#@gp :- "set title 'Radar astrometry: Goldstone (obs. epoch: 2021-06-03)'" :-
@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[-1500,500] :-
@gp :- yrange=[-5,15] :-
#@gp :- "set xtics font \"Verdana,10\"" :-
@gp :- "set xlabel '{Time-delay [\$\\mu\$s]}'" :-
@gp :- "set ylabel '{Doppler shift [Hz]}'" :-


#@gp :- "set xtics 20" :-

#@gp :- "set style fill transparent solid 0.35 noborder" :-
#@gp :- "set style circle radius $nomps" :-

@gp :- 1 ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 t '7DOF (MC sampling)'" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 t '6DOF (MC sampling)'" :-

@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' t '7DOF (3-\$\\sigma\$)'" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 t '6DOF (3-\$\\sigma\$)'" :-

#@gp :- ΔxTN_OR76.+xTN_2p5σ_ell_OR7 ΔyTN_OR76.+yTN_2p5σ_ell_OR7 "w l lw $ellw lc rgb 'red' t '7DOF (2.5-\$\\sigma\$)'" :-

@gp :- ΔxTN_OR76 ΔyTN_OR76 "w circle lc rgb 'white' fs solid 1.0 border lt -1 t '7DOF (nominal)'" :-
#@gp :- [1.1243345490830308e8].-xTN(x_OR6) [(8560/2380)*6121.1974930763245].-yTN(x_OR6) "pt 1 ps 3 lw 4 lc rgb 'red' t 'JPL 199 (nominal)'" :-

@gp :- 2 "set origin 0.31, 0.5" :-
#@gp :- "unset title" :-
@gp :- "set size 0.415, 0.45" :-
@gp :- xrange=[-11, 11] :-
@gp :- yrange=[-0.25, 0.25] :-
@gp :- "unset xlabel" :-
@gp :- "unset ylabel" :-
@gp :- "set xtics 10" :-
@gp :- "set ytics 0.1" :-

@gp :- ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 not" :-
@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not"
#@gp :- "unset multiplot"

#Gnuplot.save(term="epslatex standalone color dashed size 7in,5in font \",12\"", output="deldop_2021_Goldstone.tex")
#run(`pdflatex deldop_2021_Goldstone.tex`)

Gnuplot.options.gpviewer = false;

## April 2029 closest approach (CA) time and distance, relative to nominal OR6 orbit

In [ ]:
Gnuplot.options.gpviewer = true

jtps = 0.25
ellw = 3.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

xTN = rae_2029*au
yTN = t_2029*daysec

@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

@gp "set multiplot"
@gp :- "unset grid" :-
@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[-1800,1200] :-
@gp :- yrange=[-35,10] :-
@gp :- "set xlabel 'Geocentric range at closest approach [km]'" :-
@gp :- "set ylabel 'Time of closest approach [seconds]'" :-

#@gp :- "set xtics 20" :-

#@gp :- "set style fill transparent solid 0.35 noborder" :-
#@gp :- "set style circle radius $nomps" :-

@gp :- 1 ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 t 'OR7 (MC sampling)'" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 t 'OR6 (MC sampling)'" :-

@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' t 'OR7 3-\$\\sigma\$'" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 t 'OR6 3-\$\\sigma\$'" :-

@gp :- ΔxTN_OR76 ΔyTN_OR76 "w circle lc rgb 'white' fs solid 1.0 border lt -1 t 'OR7 nominal'" :-

@gp :- 2 "set origin 0.325, 0.125" :-
@gp :- "set size 0.4, 0.45" :-
@gp :- xrange=[-23, 23] :-
@gp :- yrange=[-1, 1] :-
@gp :- "unset xlabel" :-
@gp :- "unset ylabel" :-
@gp :- "set xtics 20" :-
@gp :- "set ytics 0.5" :-

@gp :- ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 not" :-
@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not"
#@gp :- "unset multiplot"

#Gnuplot.save(term="epslatex standalone color dashed size 7in,5in font \",12\"", output="dt_CA_2029.tex")
#run(`pdflatex dt_CA_2029.tex`)

Gnuplot.options.gpviewer = false;

In [ ]:
# multiplot example below to be run in REPL